In [50]:
import rasterio
from tqdm import tqdm
import numpy as np
import osgeo.ogr as ogr
import geopandas as gpd
import matplotlib.pyplot as plt

def read_raster(path):
    """
    reads the images using rasterio
    
    """
    source = rasterio.open(path)
    return source


def read_polygon(path):
    """
    reads the polygon
    
    """
    shapefile = gpd.read_file(path)
    return shapefile


def fractional_pixel_weights(raster_path, geometry_path):
    """
    Calculate the percent of polygon that overlaps with each pixel in the raster data.
    
    inputs : path to the raster and the polygon
    
    The general idea is that we store the coordinate from the raster for each pixel into a numpy grid.
    We later use this coordinate to create a pixel polygon similar to raster. We do this in a loop for each pixel.
    then we calculate the intersection of the raster polygon and the convex polygon.
    we also taken into consideration that there might be multiple polygons in one pixel.
    
    output : a numpy array with each percent as the values
    """
    raster_image = read_raster(raster_path)
    multi_polygon = read_polygon(geometry_path)
    
    # get the affine transformation from the raster
    gt = raster_image.transform
    
    # Create a numpy grid which has the values as coordinates of the raster.
    xs = np.arange(gt[2], gt[2] +  gt[0] * (1 + raster_image.shape[1]), gt[0])
    ys = np.arange(gt[5], gt[5] +  gt[4] * (1 + raster_image.shape[0]), gt[4])
    
    
    # Calculate the area of a single pixel
    pixel_area = abs(gt[0] * gt[4])
    
    # create a numpy array to store the percent values
    overlapping_areas = np.zeros((len(ys)-1, len(xs)-1))
    
    # looping through each polygon
    for index, row in tqdm(multi_polygon.iterrows(), total=len(multi_polygon)):
        single_polygon = row['geometry']
        
        #convert the polygon as ogr class which was previously shapely class 
        geom_ogr = ogr.CreateGeometryFromWkt(single_polygon.wkt)
        
        # we loop through the numpy array and store the coordinate to convert the pixel into polygon
        for ix in range(len(xs)-1):
            xmin = xs[ix]
            xmax = xs[ix + 1]
            for iy in range(len(ys)-1):
                ymax = ys[iy]
                ymin = ys[iy + 1]
                
                #create a 10x10 m polygon based on the coordinate of the pixel.
                coords_wkt = "POLYGON ((" + str(xmin) + ' ' + str(ymax) + ', ' + str(xmax) + ' ' + str(ymax) + ', ' + str(xmax) + ' ' + str(ymin) + ', ' + str(xmin) + ' ' + str(ymin) + ', ' + str(xmin) + ' ' + str(ymax) + "))"
                poly_cell = ogr.CreateGeometryFromWkt(coords_wkt)
                # Calculate how much the polygon intersect the pixel polygon 
                intersection = poly_cell.Intersection(geom_ogr).Area()
                overlapping_areas[iy, ix] += intersection  # Accumulate overlapping areas

    # Calculate fractional intersection
    fraction_intersected = (overlapping_areas / pixel_area) * 100

    return fraction_intersected

def save_as_raster(input_raster_path, grid, output_raster_path):
    """
    Save the grid as raster
    
    input: raster which was used to calculate the percentage, 
    the output grid which has the percent values
    path to save the file
    
    output: a raster file which contains the precent as pixel value
    """
    with rasterio.open(input_raster_path) as src:
        # Get metadata from source raster
        meta = src.meta.copy()

        # Update the metadata to reflect the number of layers
        meta.update(dtype=rasterio.float32, count=1)

        # Write output raster
        with rasterio.open(output_raster_path, 'w', **meta) as dst:
            dst.write(grid.astype(rasterio.float32), 1)


In [51]:
path_raster = "M:\\sentinel2\\braunschweig\\June\\cliped.tif"

path_geom = "M:\\lidar\\validation\\test\\braunschweig_polygon_filtered.shp"

frac_intersected = fractional_pixel_weights(path_raster, path_geom)
# Example usage
save_as_raster(path_raster, frac_intersected, 'M:\\sentinel2\\extras\\image.tif')

100%|██████████████████████████████████████████████████████████████████████████████| 1685/1685 [05:34<00:00,  5.03it/s]


In [46]:
import os
from osgeo import gdal

"""
Convert the jp2 file to tif files
"""

input_path = "M:\\sentinel2\\Brandenburg_an_der_havel\\sent\\S2B_MSIL2A_20230707T100559_N0509_R022_T33UUU_20230707T131302.SAFE\\GRANULE\\L2A_T33UUU_A033081_20230707T101428\\IMG_DATA\\combined\\"
output = "M:\\sentinel2\\extras\\"
for file in os.listdir(filename):
    if file.endswith(".jp2"):
        full_input_path = os.path.join(input_path, file)
        src_ds = gdal.Open(full_input_path)
        # Translate options - can be used to set various options like format, spatial extent, resolution etc.
        translate_options = gdal.TranslateOptions(format='GTiff',
                                          outputType=gdal.GDT_UInt16,
                                          noData=0,
                                          resampleAlg='nearest')

        # Perform the translation
        gdal.Translate(output + "{}.tif".format(file) , src_ds, options=translate_options)